In [3]:
from aiida import load_profile, orm
load_profile()

Profile<uuid='827a33dde1bc4d498af8e022f9e9dece' name='mikibonacci'>

In [4]:
from aiida_atomistic.data.structure.structure import StructureData

### Initialization and immutability

#### Just the crystal structure:

In [5]:
from ase import Atoms

unit_cell = [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]

atoms = Atoms('LiLi', [[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]], cell = [1,1,1])
atoms.set_cell(unit_cell, scale_atoms=False)
atoms.set_pbc([True,True,True])

structure = StructureData(ase=atoms)

print(structure.cell)
print(structure.sites)
print(structure.kinds)   # <== backward-compatibility
print(structure.pbc)     # <== backward-compatibility

[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
[<Site: kind name 'Li' @ 0.0,0.0,0.0>, <Site: kind name 'Li' @ 1.5,1.5,1.5>]
[<Kind: name 'Li', symbol 'Li'>]
(True, True, True)


#### Adding also properties:

In [6]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':[True,False,True]},
                },
        )

print(structure.cell)
print(structure.sites)
print(structure.pbc)

[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
[<Site: kind name 'Li' @ 0.0,0.0,0.0>, <Site: kind name 'Li' @ 1.5,1.5,1.5>]
(True, False, True)


In [5]:
structure.properties.pbc

Pbc(parent=<StructureData: uuid: a9344e20-cb24-4f46-9c6c-5faf85bf61b3 (unstored)>, value=(True, False, True))

### Immutability

In [6]:
structure.properties.pbc.value=5

TypeError: "Pbc" is immutable and does not support item assignment

In [ ]:
structure.properties =  0

AttributeError: After the initialization, `properties` is a read-only attribute

In [7]:
structure.properties.pbc = 4

NotImplementedError: After the initialization, `properties` and its attributes are read-only!

#### Trying to set some unsupported property:

In [8]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':[True,False,True]},
                'unsupported_property':{} 
        })

NotImplementedError: Property 'unsupported_property' is not yet supported.
Supported properties are: {'custom', 'pbc'}

#### Type checking for the `properties` input and its content

In [9]:
structure = StructureData(
        ase=atoms,
        properties=[('pbc',{'value':[True,False,True]})])

ValueError: The `properties` input is not of the right type. Expected '<class 'dict'>', received '<class 'list'>'.

In [10]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': [{'value':[True,False,True]}],
        })

ValueError: The 'pbc' value is not of the right type. Expected '<class 'dict'>', received '<class 'list'>'.

# Store

In [11]:
structure.store()

<StructureData: uuid: a9344e20-cb24-4f46-9c6c-5faf85bf61b3 (pk: 13767)>

In [7]:
loaded_structure = orm.load_node(13767)

In [8]:
loaded_structure.properties

AttributeError: properties

In [18]:
structure = orm.StructureData(ase=atoms)

In [25]:
structure.get_formula()

'CLi2'

## The following should not be allowed

In [21]:
structure.append_atom(position=[1,1,1],symbols="C")

## The following is a test to see if the store is correct.

However, I see that the `properties` attribute is not there, once I load again. it should be generated again in some way.
The `_property_attributes` however is there.

In [12]:
for k in loaded_structure.base.attributes.keys():
    print(k)

cell
pbc1
pbc2
pbc3
kinds
sites
_property_attributes


In [13]:
loaded_structure.base.attributes.get("_property_attributes")

{'pbc': {'value': [True, False, True]}}

In [14]:
loaded_structure.properties

AttributeError: properties